In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
corpus = pd.read_csv('./drive/My Drive/IoT_Corpus/agg_corpora.csv')
corpus = corpus.dropna()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from pprint import pprint


import matplotlib.pyplot as plt

In [4]:
def tokenization(texts):
  print(texts)
  return texts.split()

In [5]:
comments = list(corpus['Comment'].values)
texts = []
for i in comments:
  sentences = i.split(".")
  temp = []
  for j in sentences:
    temp.extend(j.split())
  texts.append(temp)

dictionary = corpora.Dictionary(texts)

In [6]:
dictionary.filter_extremes(no_below=15, no_above=0.5)

In [7]:
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]

In [8]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [18]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, 
                                       num_topics=8, 
                                       id2word=dictionary,
                                       passes=10,
                                       random_state=100,
                                       alpha=0.31,
                                       eta=0.01,
                                       per_word_topics=True)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [19]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.055*"blockchain" + 0.032*"np" + 0.031*"eclipse" + 0.028*"im" + 0.023*"drone" + 0.019*"sit" + 0.019*"websockets" + 0.015*"endpoint" + 0.015*"upgrade" + 0.014*"delay"
Topic: 1 
Words: 0.052*"thingsboard" + 0.039*"dp" + 0.032*"ref" + 0.026*"uk" + 0.025*"virtual" + 0.024*"mb" + 0.022*"manager" + 0.022*"structure" + 0.022*"nrf" + 0.022*"assistant"
Topic: 2 
Words: 0.034*"carrier" + 0.033*"certificate" + 0.027*"io" + 0.026*"error" + 0.024*"lol" + 0.023*"pycom" + 0.023*"sims" + 0.022*"cert" + 0.019*"neat" + 0.019*"luck"
Topic: 3 
Words: 0.005*"people" + 0.005*"thing" + 0.005*"security" + 0.004*"smart" + 0.004*"internet" + 0.004*"system" + 0.004*"business" + 0.004*"company" + 0.003*"technology" + 0.003*"make"
Topic: 4 
Words: 0.022*"karma" + 0.021*"bot" + 0.018*"compose" + 0.018*"motor" + 0.018*"please" + 0.017*"comment" + 0.017*"subreddit" + 0.016*"post" + 0.016*"contact" + 0.015*"reddit"
Topic: 5 
Words: 0.034*"cool" + 0.027*"name" + 0.024*"vpn" + 0.023*"bill" + 0.021*"rea

In [20]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4812766515758133


In [10]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                        id2word=dictionary,
                        num_topics=k, 
                        random_state=100,
                        passes=10,
                        alpha=a,
                        eta=b,
                        per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [12]:
import numpy as np
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
model_results = {
          'Alpha': [],
          'Beta': [],
          'Coherence': []
                }
for a in alpha:
    # iterare through beta values
  for b in beta:
      # get the coherence score for the given parameters
      cv = compute_coherence_values(corpus=corpus_tfidf, dictionary=dictionary, 
                                    k=8, a=a, b=b)
      # Save the model results
      model_results['Alpha'].append(a)
      model_results['Beta'].append(b)
      model_results['Coherence'].append(cv)


/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


ValueError: ignored

In [15]:
results = pd.DataFrame(model_results)

In [16]:
results.sort_values(by='Coherence', ascending=False)

,Alpha,Beta,Coherence
4,0.31,0.01,0.481277
3,0.01,0.91,0.469514
5,0.31,0.31,0.453500
2,0.01,0.61,0.449943
6,0.31,0.61,0.440065
7,0.31,0.91,0.426705
9,0.61,0.31,0.414199
1,0.01,0.31,0.414041
10,0.61,0.61,0.394578
0,0.01,0.01,0.391721


In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 27.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=c78bead04c251c6a668dd7312c7c88edb9b08f53e58d1a996419f8f6ceeb4c89
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=515a0465cabfa1d39d92b67b1d5bd31b99bc07ba971c067e6aceb1c215a5a5bb
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.013420  0.024296       1        1  32.184322
1      0.016663 -0.008337       2        1  28.204781
6     -0.000807 -0.006487       3        1  26.488007
5      0.024846  0.001146       4        1   4.774209
3      0.007941  0.002518       5        1   3.950918
2     -0.016152 -0.023711       6        1   2.676846
4     -0.002148  0.014026       7        1   1.177192
0     -0.016922 -0.003451       8        1   0.543725, topic_info=        Term        Freq       Total Category  logprob  loglift
368   sensor  493.000000  493.000000  Default  30.0000  30.0000
164     data  589.000000  589.000000  Default  29.0000  29.0000
666      esp  470.000000  470.000000  Default  28.0000  28.0000
50      mqtt  414.000000  414.000000  Default  27.0000  27.0000
674       io  334.000000  334.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
525     help    1.862995  271.803253   Topic8  -6.2198   0.2316
300      one    1.905533  359.153351   Topic8  -6.1973  -0.0245
6         go    1.887931  358.929230   Topic8  -6.2065  -0.0332
18   arduino    1.868326  381.392090   Topic8  -6.2170  -0.1043
70     start    1.864307  340.381531   Topic8  -6.2191   0.0073

[573 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
455       1  0.657568  accelerometer
455       2  0.149447  accelerometer
455       3  0.059779  accelerometer
455       4  0.014945  accelerometer
455       5  0.074724  accelerometer
...     ...       ...            ...
454       4  0.130520         zigbee
454       5  0.052208         zigbee
454       6  0.034805         zigbee
454       7  0.017403         zigbee
454       8  0.008701         zigbee

[2807 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 7, 6, 4, 3, 5, 1])